<a href="https://colab.research.google.com/github/HarryPotter12/PractiseML/blob/master/20_newsgroup_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [0]:
from sklearn.datasets import load_files
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib.pyplot import cm
from matplotlib import pyplot as plt
import numpy as np
import time
from sklearn import metrics

In [0]:
newsgroups_full = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [0]:
print(newsgroups_full.data[:3])

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n", 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-perf

In [0]:
X = newsgroups_full.data
y = newsgroups_full.target

In [0]:
vectorizer = TfidfVectorizer(max_features = 5000)
tfidf = vectorizer.fit_transform(X)

In [0]:
tfidf.shape

(18846, 5000)

In [0]:
vectorizer_sub = TfidfVectorizer(sublinear_tf=True, max_features = 5000)
tfidf_sub = vectorizer.fit_transform(X)

In [0]:
tfidf_sub.shape

(18846, 5000)

In [0]:
vectorizer_bigram = TfidfVectorizer(ngram_range=(1, 2), max_features = 5000)
tfidf_bigram = vectorizer_bigram.fit_transform(X)

In [0]:
tfidf_bigram.shape

(18846, 5000)

In [0]:
vectorizer_max = TfidfVectorizer(norm='max', max_features = 5000)
tfidf_max = vectorizer_max.fit_transform(X)

In [0]:
tfidf_max.shape

(18846, 5000)

for i in range(18846):
  for j in range(10000):
    if tfidf.toarray()[i][j] != tfidf_max.toarray()[i][j]:
      print("i = %d j = %d tfidf = %f tfidf_max = %f No match" % (i, j, tfidf.toarray()[i][j], tfidf_max.toarray()[i][j]))

def initialization_kmeans(X, p, q, variance_level=None):
    """
    X : dataset
    p : number of clusters
    q : dimension of the latent space
    variance_level
    pi : proportions of clusters
    mu : centers of the clusters in the observation space
    W : latent to observation matricies
    sigma2 : noise
    """

    N, d = X.shape

    # initialization
    init_centers = np.random.randint(0, N, p)
    while (len(np.unique(init_centers)) != p):
        init_centers = np.random.randint(0, N, p)

#    print(init_centers)
    mu = X[init_centers, :]
#    print("mu=", mu.shape)
    distance_square = np.zeros((N, p))
    clusters = np.zeros(N, dtype=np.int32)

    D_old = -2
    D = -1
    
    while(D_old != D):
        D_old = D

        # assign clusters
        for c in range(p):
#            print(X.shape)
#            print(mu[c, :].shape)
#            X - mu[c, :]
            distance_square[:, c] = np.power(X - mu[c, :], 2).sum(1)
        clusters = np.argmin(distance_square, axis=1)

        # compute distortion
        distmin = distance_square[range(N), clusters]
        D = distmin.sum()

        # compute new centers
        for c in range(p):
            mu[c, :] = X[clusters == c, :].mean(0)

    #for c in range(p):
    #    plt.scatter(X[clusters == c, 0], X[clusters == c, 1], c=np.random.rand(3,1))

    # parameter initialization
    pi = np.zeros(p)
    W = np.zeros((p, d, q))
    sigma2 = np.zeros(p)
    for c in range(p):
        if variance_level:
            W[c, :, :] = variance_level * np.random.randn(d, q)
        else:
            W[c, :, :] = np.random.randn(d, q)

        pi[c] = (clusters == c).sum() / N
        if variance_level:
            sigma2[c] = np.abs((variance_level/10) * np.random.randn())
        else:
            sigma2[c] = (distmin[clusters == c]).mean() / d

    return pi, mu, W, sigma2, clusters


def mppca_gem(X, pi, mu, W, sigma2, niter):
    N, d = X.shape
    p = len(sigma2)
    _, q = W[0].shape

    sigma2hist = np.zeros((p, niter))
    M = np.zeros((p, q, q))
    Minv = np.zeros((p, q, q))
    Cinv = np.zeros((p, d, d))
    logR = np.zeros((N, p))
    R = np.zeros((N, p))
    M[:] = 0.
    Minv[:] = 0.
    Cinv[:] = 0.

    L = np.zeros(niter)
    for i in range(niter):
        print('.', end='')
        for c in range(p):
            sigma2hist[c, i] = sigma2[c]

            # M
            M[c, :, :] = sigma2[c]*np.eye(q) + np.dot(W[c, :, :].T, W[c, :, :])
            Minv[c, :, :] = np.linalg.inv(M[c, :, :])

            # Cinv
            Cinv[c, :, :] = (np.eye(d)
                - np.dot(np.dot(W[c, :, :], Minv[c, :, :]), W[c, :, :].T)
                ) / sigma2[c]

            # R_ni
            deviation_from_center = X - mu[c, :]
            logR[:, c] = ( np.log(pi[c])
                + 0.5*np.log(
                    np.linalg.det(
                        np.eye(d) - np.dot(np.dot(W[c, :, :], Minv[c, :, :]), W[c, :, :].T)
                    )
                )
                - 0.5*d*np.log(sigma2[c])
                - 0.5*(deviation_from_center * np.dot(deviation_from_center, Cinv[c, :, :].T)).sum(1)
                )

        myMax = logR.max(axis=1).reshape((N, 1))
        L[i] = (
            (myMax.ravel() + np.log(np.exp(logR - myMax).sum(axis=1))).sum(axis=0)
            - N*d*np.log(2*3.141593)/2.
            )

        logR = logR - myMax - np.reshape(np.log(np.exp(logR - myMax).sum(axis=1)), (N, 1))

        myMax = logR.max(axis=0)
        logpi = myMax + np.log(np.exp(logR - myMax).sum(axis=0)) - np.log(N)
        logpi = logpi.T
        pi = np.exp(logpi)
        R = np.exp(logR)
        for c in range(p):
            mu[c, :] = (R[:, c].reshape((N, 1)) * X).sum(axis=0) / R[:, c].sum()
            deviation_from_center = X - mu[c, :].reshape((1, d))

            SW = ( (1/(pi[c]*N))
                * np.dot((R[:, c].reshape((N, 1)) * deviation_from_center).T,
                    np.dot(deviation_from_center, W[c, :, :]))
                )

            Wnew = np.dot(SW, np.linalg.inv(sigma2[c]*np.eye(q) + np.dot(np.dot(Minv[c, :, :], W[c, :, :].T), SW)))

            sigma2[c] = (1/d) * (
                (R[:, c].reshape(N, 1) * np.power(deviation_from_center, 2)).sum()
                /
                (N*pi[c])
                -
                np.trace(np.dot(np.dot(SW, Minv[c, :, :]), Wnew.T))
                )

            W[c, :, :] = Wnew

    return pi, mu, W, sigma2, R, L, sigma2hist


def mppca_predict(X, pi, mu, W, sigma2):
    N, d = X.shape
    p = len(sigma2)
    _, q = W[0].shape

    M = np.zeros((p, q, q))
    Minv = np.zeros((p, q, q))
    Cinv = np.zeros((p, d, d))
    logR = np.zeros((N, p))
    R = np.zeros((N, p))

    for c in range(p):
        # M
        M[c, :, :] = sigma2[c] * np.eye(q) + np.dot(W[c, :, :].T, W[c, :, :])
        Minv[c, :, :] = np.linalg.inv(M[c, :, :])

        # Cinv
        Cinv[c, :, :] = (np.eye(d)
            - np.dot(np.dot(W[c, :, :], Minv[c, :, :]), W[c, :, :].T)
            ) / sigma2[c]

        # R_ni
        deviation_from_center = X - mu[c, :]
        logR[:, c] = ( np.log(pi[c])
            + 0.5*np.log(
                np.linalg.det(
                    np.eye(d) - np.dot(np.dot(W[c, :, :], Minv[c, :, :]), W[c, :, :].T)
                )
            )
            - 0.5*d*np.log(sigma2[c])
            - 0.5*(deviation_from_center * np.dot(deviation_from_center, Cinv[c, :, :].T)).sum(1)
            )

    myMax = logR.max(axis=1).reshape((N, 1))
    logR = logR - myMax - np.reshape(np.log(np.exp(logR - myMax).sum(axis=1)), (N, 1))
    R = np.exp(logR)

    return R

p = 20 # We expect 2 clusters
q = 1 # We expect 1-dimensional subspaces
t = time.time()
pi, mu, W, sigma2, clusters = initialization_kmeans(tfidf.toarray(), p, q)
print("Elapsed :", time.time()-t, "seconds")

niter = 500
t = time.time()
pi, mu, W, sigma2, R, L, sigma2hist = mppca_gem(tfidf.toarray(), pi, mu, W, sigma2, niter)
print("Elapsed :", time.time()-t, "seconds")

clusters = R.argmax(axis=1)
#print_clusters(X, clusters, mu)

In [0]:
from sklearn.mixture import GaussianMixture

In [0]:
gmm = GaussianMixture(n_components=20)
gmm.fit(tfidf.toarray())
# predict the cluster for each data point
y_pred = gmm.predict(tfidf.toarray())
y_pred

array([17,  2,  8, ..., 17,  8,  4])

gammas = [0.01, 0.1, 1, 10, 100]

best_sil = float("-inf")
best_gamma = 1000
for my_gamma in gammas:
#  print("For gamma = ", my_gamma)
  km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=my_gamma)
  y_pred = km.fit_predict(tfidf)
#  print("Computing silhouette_score ")
  silhouette = metrics.silhouette_score(tfidf, y_pred)
  print("The silhouette_score for gamma = %f is = %.4f" % (gamma, silhouette))
  if silhouette > best_sil:
#    print("Old Sil : ", best_sil)
    best_sil = silhouette
    best_gamma = my_gamma
#    print("New Sil : ", best_sil, "gamma : ", best_gamma)

print("best_gamma : ", best_gamma)
km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=best_gamma)
y_pred = km.fit_predict(tfidf)

In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.018
NMI: 0.066
AMI: 0.043
Homogeneity: 0.045
Completeness: 0.097
V-measure: 0.061
FMI: 0.141


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
gmm = GaussianMixture(n_components=20)
gmm.fit(tfidf_sub.toarray())
# predict the cluster for each data point
y_pred = gmm.predict(tfidf_sub.toarray())
y_pred

array([11,  8, 17, ..., 11, 11, 17])

In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.014
NMI: 0.057
AMI: 0.033
Homogeneity: 0.035
Completeness: 0.094
V-measure: 0.050
FMI: 0.153


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
gmm = GaussianMixture(n_components=20)
gmm.fit(tfidf_max.toarray())
# predict the cluster for each data point
y_pred = gmm.predict(tfidf_max.toarray())
y_pred

array([ 7,  3,  1, ...,  7,  4, 12])

In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.021
NMI: 0.124
AMI: 0.093
Homogeneity: 0.095
Completeness: 0.161
V-measure: 0.120
FMI: 0.130


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
gmm = GaussianMixture(n_components=20)
gmm.fit(tfidf_bigram.toarray())
# predict the cluster for each data point
y_pred = gmm.predict(tfidf_bigram.toarray())
y_pred

array([16,  4,  1, ..., 16, 16,  1])

In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.013
NMI: 0.053
AMI: 0.031
Homogeneity: 0.032
Completeness: 0.089
V-measure: 0.047
FMI: 0.152


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
